**ETL Creacion Dimensiones**

***1. Creación Dim_Fecha_Entrada***

In [1]:
import mysql.connector
from mysql.connector import errorcode
from datetime import datetime, timedelta

# Datos de conexión
config = {
    'user': 'dmonroy3',
    'password': 'SamuelAlejo2020%',
    'host': 'localhost',
    'database': 'paad_g17'
}

# Script SQL para eliminar la tabla si existe
drop_table_script = """
DROP TABLE IF EXISTS Dim_Fecha_Entrada;
"""

# Script SQL para crear la tabla
create_table_script = """
CREATE TABLE Dim_Fecha_Entrada (
    IDFecha INT PRIMARY KEY,
    Fecha DATE,
    ID_Año INT,
    ID_Mes INT,
    Mes VARCHAR(15),
    ID_Semana_Año INT,
    ID_Semana_Mes INT,
    ID_Dia_Año INT,
    ID_Dia_Mes INT,
    ID_Dia_Semana INT,
    Dia_Semana VARCHAR(15),
    ID_Periodo INT
);
"""

# Función para insertar datos en la tabla
def populate_table(cursor):
    insert_data_script = """
    INSERT INTO Dim_Fecha_Entrada (
        IDFecha, Fecha, ID_Año, ID_Mes, Mes, ID_Semana_Año, ID_Semana_Mes, ID_Dia_Año, 
        ID_Dia_Mes, ID_Dia_Semana, Dia_Semana, ID_Periodo
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    current_date = datetime.strptime('2022-01-01', '%Y-%m-%d')
    while current_date <= datetime.strptime('2023-12-31', '%Y-%m-%d'):
        cursor.execute(insert_data_script, (
            int(current_date.strftime('%Y%m%d')),
            current_date.strftime('%Y-%m-%d'),
            int(current_date.strftime('%Y')),
            int(current_date.strftime('%m')),
            current_date.strftime('%B'),
            int(current_date.strftime('%U')),
            int(current_date.strftime('%U')) - int(current_date.replace(day=1).strftime('%U')) + 1,
            int(current_date.strftime('%j')),
            int(current_date.strftime('%d')),
            int(current_date.strftime('%w')) + 1,
            current_date.strftime('%A'),
            int(current_date.strftime('%Y%m'))
        ))
        current_date += timedelta(days=1)

# Conexión a la base de datos
try:
    cnx = mysql.connector.connect(**config)
    cursor = cnx.cursor()

    # Eliminar la tabla si existe
    cursor.execute(drop_table_script)
    print("Tabla Dim_Fecha_Entrada eliminada si existía.")

    # Crear la tabla
    cursor.execute(create_table_script)
    print("Tabla Dim_Fecha_Entrada creada correctamente.")

    # Insertar datos en la tabla
    populate_table(cursor)
    cnx.commit()
    print("Datos insertados correctamente.")

except mysql.connector.Error as err:
    print(f"Error: {err}")
finally:
    cursor.close()
    cnx.close()

Tabla Dim_Fecha_Entrada eliminada si existía.
Tabla Dim_Fecha_Entrada creada correctamente.
Datos insertados correctamente.


***2. Creación Dim_Proveedor***

In [2]:
import mysql.connector

# Conectarse a MySQL
cnx = mysql.connector.connect(
    host='localhost', 
    user='dmonroy3',  
    password='SamuelAlejo2020%',  
    database='paad_g17'  
)

cursor = cnx.cursor()

# Dropear la tabla Dim_Proveedor si existe
drop_table_query = "DROP TABLE IF EXISTS Dim_Proveedor"
cursor.execute(drop_table_query)

# Crear la tabla Dim_Proveedor con ID_Proveedor como clave primaria autoincremental
create_table_query = """
CREATE TABLE Dim_Proveedor (
    ID_Proveedor INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
    Proveedor VARCHAR(255) NOT NULL,
    Seg_Proveedor VARCHAR(25)
) AUTO_INCREMENT=10
"""
cursor.execute(create_table_query)

# Insertar los supplier_id únicos desde la tabla stg_baq_compras con limpieza
insert_query = """
INSERT INTO Dim_Proveedor (Proveedor)
SELECT DISTINCT TRIM(supplier_id)
FROM stg_baq_compras
"""

cursor.execute(insert_query)

# Confirmar los cambios
cnx.commit()

# Obtener el número de registros insertados
cursor.execute("SELECT COUNT(*) FROM Dim_Proveedor")
num_records = cursor.fetchone()[0]

# Imprimir el número de registros insertados
print(f"Se han insertado {num_records} registros en la tabla Dim_Proveedor.")

# Cerrar la conexión
cursor.close()
cnx.close()


Se han insertado 141 registros en la tabla Dim_Proveedor.


***3. Creación Dim_Locacion***

In [3]:
import mysql.connector

# Conectarse a MySQL
cnx = mysql.connector.connect(
    host='localhost', 
    user='dmonroy3',  
    password='SamuelAlejo2020%',  
    database='paad_g17' 
)

cursor = cnx.cursor()

# Dropear la tabla Dim_Locacion si existe
drop_table_query = "DROP TABLE IF EXISTS Dim_Locacion"
cursor.execute(drop_table_query)

# Crear la tabla Dim_Locacion con ID_Locacion como clave primaria autoincremental
create_table_query = """
CREATE TABLE Dim_Locacion (
    ID_Locacion INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
    ID_Bodega VARCHAR(100) NOT NULL,
    ID_Region VARCHAR(100) NOT NULL,
    ID_Departamento VARCHAR(100) NOT NULL,
    Departamento VARCHAR(100) NOT NULL,
    ID_Municipio VARCHAR(100) NOT NULL,
    Municipio VARCHAR(100) NOT NULL
) AUTO_INCREMENT=100
"""
cursor.execute(create_table_query)

# Insertar los datos en la tabla Dim_Locacion
insert_query = """
INSERT INTO Dim_Locacion (ID_Bodega, ID_Region, ID_Departamento, Departamento, ID_Municipio, Municipio)
SELECT DISTINCT 
    TRIM(warehouse_code),
    TRIM(region_code),
    '08',
    'Atlantico',
    '001',
    'Barranquilla'
FROM stg_baq_compras
"""

cursor.execute(insert_query)

# Confirmar los cambios
cnx.commit()

# Obtener el número de registros insertados
cursor.execute("SELECT COUNT(*) FROM Dim_Locacion")
num_records = cursor.fetchone()[0]

# Imprimir el número de registros insertados
print(f"Se han insertado {num_records} registros en la tabla Dim_Locacion.")

# Cerrar la conexión
cursor.close()
cnx.close()


Se han insertado 1 registros en la tabla Dim_Locacion.


***4. Creación Bridge_Duracion***

In [4]:
import mysql.connector

# Conectarse a MySQL
cnx = mysql.connector.connect(
    host='localhost', 
    user='dmonroy3',  
    password='SamuelAlejo2020%',  
    database='paad_g17' 
)
cursor = cnx.cursor()

# Dropear la tabla bridge_duracion si existe
drop_table_query = "DROP TABLE IF EXISTS bridge_duracion"
cursor.execute(drop_table_query)

# Crear la tabla bridge_duracion con una columna Duracion
create_table_query = """
CREATE TABLE bridge_duracion (
    ID_shelf_life VARCHAR(100) NOT NULL
)
"""
cursor.execute(create_table_query)

# Insertar los distintos valores de shelf_life en la tabla bridge_duracion
insert_query = """
INSERT INTO bridge_duracion (ID_shelf_life)
SELECT DISTINCT shelf_life
FROM stg_waste_percentage_by_age
"""

cursor.execute(insert_query)

# Confirmar los cambios
cnx.commit()

# Obtener el número de registros insertados
cursor.execute("SELECT COUNT(*) FROM bridge_duracion")
num_records = cursor.fetchone()[0]

# Imprimir el número de registros insertados
print(f"Se han insertado {num_records} registros en la tabla bridge_duracion.")

# Cerrar la conexión
cursor.close()
cnx.close()


Se han insertado 89 registros en la tabla bridge_duracion.


***5. Creación Dim_Duracion***

In [5]:
import mysql.connector

# Conectarse a MySQL
cnx = mysql.connector.connect(
    host='localhost', 
    user='dmonroy3',  
    password='SamuelAlejo2020%',  
    database='paad_g17' 
)

cursor = cnx.cursor()

# Dropear la tabla dim_duracion si existe
drop_table_query = "DROP TABLE IF EXISTS dim_duracion"
cursor.execute(drop_table_query)

# Crear la tabla dim_duracion con las columnas especificadas
create_table_query = """
CREATE TABLE dim_duracion (
    ID_shelf_life VARCHAR(100) NOT NULL,
    edad INT NOT NULL,
    porcentaje_perdida DECIMAL(10,2) NOT NULL
)
"""
cursor.execute(create_table_query)

# Insertar los valores desde la tabla stg_waste_percentage_by_age
insert_query = """
INSERT INTO dim_duracion (ID_shelf_life, edad, porcentaje_perdida)
SELECT shelf_life AS ID_shelf_life, age AS edad, waste_percentage / 100.0 AS porcentaje_perdida
FROM stg_waste_percentage_by_age
"""

cursor.execute(insert_query)

# Confirmar los cambios
cnx.commit()

# Obtener el número de registros insertados
cursor.execute("SELECT COUNT(*) FROM dim_duracion")
num_records = cursor.fetchone()[0]

# Imprimir el número de registros insertados
print(f"Se han insertado {num_records} registros en la tabla dim_duracion.")

# Cerrar la conexión
cursor.close()
cnx.close()


Se han insertado 7921 registros en la tabla dim_duracion.


***6. Creación Dim_Producto***

In [6]:
import mysql.connector

# Conectarse a MySQL
cnx = mysql.connector.connect(
    host='localhost', 
    user='dmonroy3',  
    password='SamuelAlejo2020%',  
    database='paad_g17' 
)

cursor = cnx.cursor()

# Dropear la tabla Dim_Producto si existe
drop_table_query = "DROP TABLE IF EXISTS Dim_Producto"
cursor.execute(drop_table_query)

# Crear la tabla Dim_Producto con las columnas especificadas
create_table_query = """
CREATE TABLE Dim_Producto (
    ID_Producto VARCHAR(100) NOT NULL,
    SKU_Producto VARCHAR(100) NOT NULL,
    Producto VARCHAR(100) NOT NULL,
    ID_Categoria VARCHAR(100) NOT NULL,
    Categoria VARCHAR(100) NOT NULL,
    Mean_shelf_life INT,
    Tiempo_Promedio_Entrega DECIMAL(10,2),
    Unidad_Compra DECIMAL(10,2),
    Unidad_medida_Compra VARCHAR(100)
)
"""
cursor.execute(create_table_query)

# Insertar los valores desde la tabla stg_products_baq
insert_query = """
INSERT INTO Dim_Producto (ID_Producto, SKU_Producto, Producto, ID_Categoria, Categoria, Mean_shelf_life, Tiempo_Promedio_Entrega, Unidad_Compra, Unidad_medida_Compra)
SELECT 
    TRIM(product_id),
    TRIM(sku),
    TRIM(name),
    TRIM(product_category_id),
    TRIM(category),
    mean_shelf_life,
    promised_lead_time,
    purchasing_unit,
    buy_unit
FROM stg_products_baq
"""

cursor.execute(insert_query)

# Confirmar los cambios
cnx.commit()

# Obtener el número de registros insertados
cursor.execute("SELECT COUNT(*) FROM Dim_Producto")
num_records = cursor.fetchone()[0]

# Imprimir el número de registros insertados
print(f"Se han insertado {num_records} registros en la tabla Dim_Producto.")

# Cerrar la conexión
cursor.close()
cnx.close()


Se han insertado 137 registros en la tabla Dim_Producto.
